# LG VS 두산 잠실의 주인은 누구인가?  

2015년부터 2021년까지의 데이터분석을 통한 LG트윈스 두산베어스 구단 비교분석  

가설 
LG가 잠실의 주인이다
TRUE 1을 COUNT해 그 수가 과반이 넘을 경우, 가설 채택 
그렇지 않을 경우, 가설 기각 = "두산이 잠실의 주인이다" 채택 

검증 변수 
1.	(LG-두산을 제외한) 잠실(홈)에서의 승률: LG 승률 > 두산의 승률 T / F
    * KBO는 2011년부터 승률 게산 방식에서 무승부를 카운팅하지 않는다. 즉 승률은 
승수 / 전체경기수 - 무승부 경기 수 가 된다.  


2.	(LG VS 두산) 잠실에서의 승률: LG 승률 > 두산의 승률 T / F
3.	잠실에서의 평균관중수: LG 평균관중수 > 두산의 평균 관중수 T / F
4.	잠실에서의 우천경기시 평균관중수 (팬 결집도, 충성도 측정): LG 평균관중수 > 두산의 평균 관중수 T / F
5.	올스타 맴버 수: LG > 두산
6.	(시간이 되면 진행) 구단 관련 부정기사 수: LG < 두산 T / F
7.	(시간이 되면 진행) 구단 관련 긍정기사 수: LG > 두산 T / F

검증 결과 
5 항목 (혹은 7항목)의 True 값을 count 해서 절반 이상의 ex. T: 3, F: 2 결과인 경우 가설을 채택  
  
2번 재용 / 3번 다혜 / 4번 준희 / 5번 준행

In [27]:
import numpy as np
import pandas as pd
import time 
import requests 
from bs4 import BeautifulSoup  

In [28]:
# 1번 조건에 대한 두산 데이터 집합 
# 두산의 2015-2021 row data 크롤링 
info = {
    'Referer': 'http://old.statiz.co.kr/team.php?opt=0&sopt=1&year=2015&team=%EB%91%90%EC%82%B0',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
}
df_D_row = pd.DataFrame()
for season in range(2015, 2022):
    url = f'http://old.statiz.co.kr/team.php?opt=0&sopt=1&year={season}&team=%EB%91%90%EC%82%B0'

    df_temp = pd.read_html(url, header=1)[0]
    df_temp['연도'] = season
    df_temp['홈팀'] = '두산'
    df_D_row = pd.concat([df_D_row, df_temp], ignore_index=True)
df_D_row

# LG 팀 제거
D_lg_select = df_D_row[df_D_row['상대팀'] == 'LG'].index
D_lg_remove = df_D_row.drop(D_lg_select)
D_lg_remove

# & 시즌 정규, 구장 잠실 필터링
DS1 = D_lg_remove[(D_lg_remove["시즌"] == "정규") & (D_lg_remove["구장"] == "잠실")]
DS1

,시즌,날짜,시간,구장,팀 선발,T,결과,상대팀,상대선발,T.1,승리투수,패전투수,세이브,관중,성적,연도,홈팀
0,정규,03-28,14시,잠실,마야,R,W 9:4,NC,찰리,R,마야,임정호,NaN,21746.0,1-0-0,2015,두산
1,정규,03-29,14시,잠실,장원준,L,W 4:1,NC,손민한,R,장원준,손민한,윤명준,15814.0,2-0-0,2015,두산
8,정규,04-07,18시30분,잠실,유희관,L,L 4:17,넥센,피어밴드,L,피어밴드,유희관,NaN,8038.0,3-4-0,2015,두산
9,정규,04-08,18시30분,잠실,진야곱,L,W 9:4,넥센,김대우,RU,진야곱,김대우,NaN,7962.0,4-4-0,2015,두산
10,정규,04-09,18시30분,잠실,마야,R,W 1:0,넥센,밴헤켄,L,마야,밴헤켄,NaN,8226.0,5-4-0,2015,두산
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143,정규,10-17,14시,잠실,곽빈,NaN,D 3:3,KIA,이민우,NaN,NaN,NaN,NaN,NaN,64-62-6,2021,두산
1144,정규,10-17,17시,잠실,현도훈,R,W 5:2,KIA,윤중현,NaN,이현승,윤중현,이영하,NaN,65-62-6,2021,두산
1151,정규,10-26,18시30분,잠실,최원준,R,W 7:2,키움,최원태,R,이현승,최원태,NaN,2293.0,68-64-8,2021,두산
1156,정규,11-01,18시30분,잠실,곽빈,NaN,L 4:7,키움,안우진,NaN,조상우,김강률,NaN,12422.0,71-66-8,2021,두산


In [29]:
# 1번 조건에 대한 LG 데이터 집합 
# LG의 2015-2021 row data 크롤링 
info = {
    'Referer': 'http://old.statiz.co.kr/team.php?opt=0&sopt=1&year=2015&team=LG',
                
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'               
}
df_L_row = pd.DataFrame()

for season in range(2015, 2022):
    url = f'http://old.statiz.co.kr/team.php?opt=0&sopt=1&year={season}&team=LG'
            
    df_temp = pd.read_html(url, header=1)[0]
    df_temp['연도'] = season
    df_temp['홈팀'] = "LG"
    df_L_row = pd.concat([df_L_row, df_temp], ignore_index=True)
    
df_L_row

#정규, 잠실, 두산제외 
LS1 = df_L_row[(df_L_row['시즌']=='정규') & (df_L_row['구장']=='잠실') & (df_L_row['상대팀']!='두산')]
LS1

,시즌,날짜,시간,구장,팀 선발,T,결과,상대팀,상대선발,T.1,승리투수,패전투수,세이브,관중,성적,연도,홈팀
2,정규,03-31,18시30분,잠실,루카스,R,L 1:7,롯데,린드블럼,R,린드블럼,루카스,NaN,NaN,0-3-0,2015,LG
3,정규,04-01,18시30분,잠실,임정우,R,W 3:2,롯데,이상화,R,이동현,이정민,NaN,14260.0,1-3-0,2015,LG
4,정규,04-02,18시30분,잠실,NaN,NaN,우천,롯데,NaN,NaN,NaN,NaN,NaN,NaN,1-3-0,2015,LG
5,정규,04-03,18시30분,잠실,소사,R,L 3:7,삼성,클로이드,R,안지만,정찬헌,NaN,12636.0,1-4-0,2015,LG
6,정규,04-04,17시,잠실,임지섭,L,W 3:2,삼성,피가로,R,임지섭,피가로,봉중근,20404.0,2-4-0,2015,LG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,정규,10-11,14시,잠실,이민호,NaN,L 2:4,KT,데스파이네,NaN,데스파이네,함덕주,김재윤,NaN,66-52-7,2021,LG
1114,정규,10-19,18시30분,잠실,임찬규,R,L 4:5,키움,안우진,NaN,안우진,임찬규,김태훈,1624.0,69-55-9,2021,LG
1115,정규,10-20,18시30분,잠실,켈리,NaN,L 5:6,키움,최원태,R,최원태,켈리,김태훈,1606.0,69-56-9,2021,LG
1116,정규,10-21,18시30분,잠실,임준형,NaN,D 5:5,키움,정찬헌,R,NaN,NaN,NaN,1617.0,69-56-10,2021,LG


In [30]:
#연도별로 돌아가며 각 팀 승률계산 (이미 홈, 정규시즌, 잠실, 상대팀 제외 조건이 적용)
df_win_rate = pd.DataFrame(index = range(2015,2022),columns = ['두산 홈 승률','LG 홈 승률'])
j = 0
for i in range(2015,2022):
    df_win_rate.iloc[j]['두산 홈 승률'] = sum(DS1[DS1['연도']==i]['결과'].str.contains('W.'))/(sum(DS1[DS1['연도']==i]['결과'].str.contains('W.')) + sum(DS1[DS1['연도']==i]['결과'].str.contains('L.')))
    df_win_rate.iloc[j]['LG 홈 승률'] = sum(LS1[LS1['연도']==i]['결과'].str.contains('W.'))/(sum(LS1[LS1['연도']==i]['결과'].str.contains('W.')) + sum(LS1[LS1['연도']==i]['결과'].str.contains('L.')))
    j = j + 1
    #print (i)b
df_win_rate['주인여부'] = df_win_rate['두산 홈 승률'] < df_win_rate['LG 홈 승률']
df_win_rate

,두산 홈 승률,LG 홈 승률,주인여부
2015,0.578125,0.467742,False
2016,0.666667,0.587302,False
2017,0.596774,0.578125,False
2018,0.6875,0.619048,False
2019,0.640625,0.587302,False
2020,0.596774,0.596774,False
2021,0.52381,0.590164,True


In [8]:
#1번 승률 계산 전 개략적 아이디어 (1)
df_win_rate = pd.DataFrame(index = range(2015,2022),columns = ['두산 승률','LG 승률'])
df_win_rate

,두산 승률,LG 승률
2015,NaN,NaN
2016,NaN,NaN
2017,NaN,NaN
2018,NaN,NaN
2019,NaN,NaN
2020,NaN,NaN
2021,NaN,NaN


In [31]:
#1번 승률 계산 전 개략적 아이디어 (2)
for i in range(2015,2022):
    DS1[DS1["연도"] == i].sum(D_2015['결과'].str.contains('W.')) / ()
    DS1[DS1["연도"] == i]
    
D_hw_2015 = sum(D_2015['결과'].str.contains('W.'))
D_hl_2015 = sum(D_2015['결과'].str.contains('L.'))
D_wp_2015  = D_hw_2015 / (D_hw_2015 + D_hl_2015)

TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [4]:
#2015부터 2021까지 LG row data
info = {
    'Referer': 'http://old.statiz.co.kr/team.php?opt=0&sopt=1&year=2015&team=LG',
                
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'               
}
df_L_row = pd.DataFrame()

for season in range(2015, 2022):
    url = f'http://old.statiz.co.kr/team.php?opt=0&sopt=1&year={season}&team=LG'
            
    df_temp = pd.read_html(url, header=1)[0]
    df_temp['연도'] = season
    df_temp['홈팀'] = "LG"
    df_L_row = pd.concat([df_L_row, df_temp], ignore_index=True)
    
df_L_row

#정규, 잠실, 두 선제조건 
LS1 = df_L_row[(df_L_row['시즌']=='정규') & (df_L_row['구장']=='잠실') & (df_L_row['상대팀']!='두산')]
LS1


#LG 2015년 (상대팀 두산 제외) 홈 경기 데이터 
df_LG_2015 = df_LG정규잠실두산제외[df_LG정규잠실두산제외['연도']==2015]
df_LG_2015

#2015시즌 LG (상대팀 두산 제외) 홈 승률
L_hw_2015 = sum(df_LG_2015['결과'].str.contains('W.'))
#L_hw_2015 = 29 승
L_hl_2015 = sum(df_LG_2015['결과'].str.contains('L.'))
#L_hl_2015 = 33 패

L_wp_2015 = L_hw_2015 / (L_hw_2015 + L_hl_2015)

L_wp_2015


0.46774193548387094

In [21]:
LS1 = df_L_row[(df_L_row['시즌']=='정규') & (df_L_row['구장']=='잠실') & (df_L_row['상대팀']!='두산')]
# df[df['결과'].str.contains('W.')]
LS1

# df_LG정규잠실두산제외.to_csv('C:/Users/user/Desktop/mini_proj/whocare.ipynb.csv', encoding='cp949')

,시즌,날짜,시간,구장,팀 선발,T,결과,상대팀,상대선발,T.1,승리투수,패전투수,세이브,관중,성적,연도,홈팀
2,정규,03-31,18시30분,잠실,루카스,R,L 1:7,롯데,린드블럼,R,린드블럼,루카스,NaN,NaN,0-3-0,2015,LG
3,정규,04-01,18시30분,잠실,임정우,R,W 3:2,롯데,이상화,R,이동현,이정민,NaN,14260.0,1-3-0,2015,LG
4,정규,04-02,18시30분,잠실,NaN,NaN,우천,롯데,NaN,NaN,NaN,NaN,NaN,NaN,1-3-0,2015,LG
5,정규,04-03,18시30분,잠실,소사,R,L 3:7,삼성,클로이드,R,안지만,정찬헌,NaN,12636.0,1-4-0,2015,LG
6,정규,04-04,17시,잠실,임지섭,L,W 3:2,삼성,피가로,R,임지섭,피가로,봉중근,20404.0,2-4-0,2015,LG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1106,정규,10-11,14시,잠실,이민호,NaN,L 2:4,KT,데스파이네,NaN,데스파이네,함덕주,김재윤,NaN,66-52-7,2021,LG
1114,정규,10-19,18시30분,잠실,임찬규,R,L 4:5,키움,안우진,NaN,안우진,임찬규,김태훈,1624.0,69-55-9,2021,LG
1115,정규,10-20,18시30분,잠실,켈리,NaN,L 5:6,키움,최원태,R,최원태,켈리,김태훈,1606.0,69-56-9,2021,LG
1116,정규,10-21,18시30분,잠실,임준형,NaN,D 5:5,키움,정찬헌,R,NaN,NaN,NaN,1617.0,69-56-10,2021,LG


In [9]:
#LG 2015년 (상대팀 두산 제외) 홈 경기 데이터 
df_LG_2015 = df_LG정규잠실두산제외[df_LG정규잠실두산제외['연도']==2015]
df_LG_2015

# df_LG_2015.to_csv('C:/Users/user/Desktop/mini_proj/whocare.ipynb.csv', encoding='cp949')

,시즌,날짜,시간,구장,팀 선발,T,결과,상대팀,상대선발,T.1,승리투수,패전투수,세이브,관중,성적,연도,홈팀
2,정규,03-31,18시30분,잠실,루카스,R,L 1:7,롯데,린드블럼,R,린드블럼,루카스,NaN,NaN,0-3-0,2015,LG
3,정규,04-01,18시30분,잠실,임정우,R,W 3:2,롯데,이상화,R,이동현,이정민,NaN,14260.0,1-3-0,2015,LG
4,정규,04-02,18시30분,잠실,NaN,NaN,우천,롯데,NaN,NaN,NaN,NaN,NaN,NaN,1-3-0,2015,LG
5,정규,04-03,18시30분,잠실,소사,R,L 3:7,삼성,클로이드,R,안지만,정찬헌,NaN,12636.0,1-4-0,2015,LG
6,정규,04-04,17시,잠실,임지섭,L,W 3:2,삼성,피가로,R,임지섭,피가로,봉중근,20404.0,2-4-0,2015,LG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,정규,09-21,18시30분,잠실,류제국,R,L 1:4,kt,옥스프링,R,옥스프링,진해수,조무근,6560.0,58-75-2,2015,LG
152,정규,09-28,14시,잠실,우규민,RU,W 8:4,KIA,김광수,R,우규민,김광수,NaN,23365.0,62-75-2,2015,LG
154,정규,10-01,18시30분,잠실,소사,R,L 2:7,NC,해커,R,해커,소사,임창민,6401.0,62-77-2,2015,LG
155,정규,10-02,18시30분,잠실,루카스,R,L 3:6,한화,안영명,R,안영명,루카스,송은범,16252.0,62-78-2,2015,LG


In [10]:
#2015시즌 LG (상대팀 두산 제외) 홈 승률
L_hw_2015 = sum(df_LG_2015['결과'].str.contains('W.'))
#L_hw_2015 = 29 승
L_hl_2015 = sum(df_LG_2015['결과'].str.contains('L.'))
#L_hl_2015 = 33 패

L_wp_2015 = L_hw_2015 / (L_hw_2015 + L_hl_2015)

round(L_wp_2015, 3)

0.468

In [13]:
L_wp_2015 > D_wp_2015

False

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   시즌      158 non-null    object 
 1   날짜      158 non-null    object 
 2   시간      158 non-null    object 
 3   구장      158 non-null    object 
 4   팀 선발    145 non-null    object 
 5   T       145 non-null    object 
 6   결과      158 non-null    object 
 7   상대팀     158 non-null    object 
 8   상대선발    145 non-null    object 
 9   T.1     145 non-null    object 
 10  승리투수    142 non-null    object 
 11  패전투수    142 non-null    object 
 12  세이브     54 non-null     object 
 13  관중      140 non-null    float64
 14  성적      158 non-null    object 
dtypes: float64(1), object(14)
memory usage: 18.6+ KB


In [13]:
# (연습) LG 2015년 시즌 승률 (홈+어웨이)
L_win_2015 = sum(df['결과'].str.contains('W.'))
L_lose_2015 = sum(df['결과'].str.contains('L.'))

L_winpercentage_2015 = (L_win_2015 / (L_win_2015 + L_lose_2015))
L_winpercentage_2015

0.4507042253521127

In [7]:
L_nm_2015 = df['관중'].mean()
# round(L_nm_2015)
L_nm_2015

12541.85

In [36]:
#홈 승률
# df[(df['구장']=='잠실') & (df['결과'].str.contains('W.'))]
# len(df[(df['구장']=='잠실') & (df['결과'].str.contains('W.'))])

# df[(df['구장']=='잠실') & (df['결과'].str.contains('L.'))]
# len(df[(df['구장']=='잠실') & (df['결과'].str.contains('L.'))])

228

In [17]:
df = pd.DataFrame(index=[2015, 2016, 2017, 2018, 2019])

In [18]:
df['L'] = [1, 2, 3, 4, 5]
df['D'] = [2, 3, 4, 1, 2]

In [21]:
df['W'] = df['L'] > df['D']

In [23]:
df['W'].sum()

2

In [6]:
df_D_row[df_D_row['연도'] == 2015]

,시즌,날짜,시간,구장,팀 선발,T,결과,상대팀,상대선발,T.1,승리투수,패전투수,세이브,관중,성적,연도,홈팀
0,정규,03-28,14시,잠실,마야,R,W 9:4,NC,찰리,R,마야,임정호,NaN,21746.0,1-0-0,2015,두산
1,정규,03-29,14시,잠실,장원준,L,W 4:1,NC,손민한,R,장원준,손민한,윤명준,15814.0,2-0-0,2015,두산
2,정규,03-31,18시30분,@대전,NaN,NaN,우천,한화,NaN,NaN,NaN,NaN,NaN,NaN,2-0-0,2015,두산
3,정규,04-01,18시30분,@대전한밭,유희관,L,W 6:3,한화,유먼,L,유희관,유먼,윤명준,6554.0,3-0-0,2015,두산
4,정규,04-02,18시30분,@대전,진야곱,L,L 2:4,한화,탈보트,R,탈보트,진야곱,윤규진,3756.0,3-1-0,2015,두산
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,한국시리즈,10-26,18시30분,@대구,유희관,L,L 8:9,삼성,피가로,R,백정현,이현승,차우찬,10000.0,0-1-0,2015,두산
173,한국시리즈,10-27,18시30분,@대구,니퍼트,R,W 6:1,삼성,장원삼,L,니퍼트,장원삼,NaN,10000.0,1-1-0,2015,두산
174,한국시리즈,10-29,18시30분,잠실,장원준,L,W 5:1,삼성,클로이드,R,장원준,클로이드,NaN,25000.0,2-1-0,2015,두산
175,한국시리즈,10-30,18시30분,잠실,이현호,L,W 4:3,삼성,피가로,R,노경은,피가로,이현승,25000.0,3-1-0,2015,두산
